# Assignment 2 - Spark Dataframes
***Note***: All the dataset files were stored in the same folder as this notebook.

In [1]:
import os
import pyspark
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
spark

## 5. 25 Points
**Dataset**:  populationbycountry19802010millions.csv

**Solve**: For region, compute the ***percentage change*** in population, year over year.  Note the year 1980 will not have a preceding year. For each year, display the region with the top population ***decrease***.

**Example** (not actual results):

1981,  North America, -2%

1982, Aruba, -7%…

### Assumptions:
The region with the top population ***decrease*** has the lowest ***percentage change***.

How to make unions: https://stackoverflow.com/questions/52685609/add-new-rows-to-pyspark-dataframe

In [2]:
from pyspark.sql.functions import first, desc, lit, col
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType
population = spark.read.option("header", True).option("InferSchema", True).csv("populationbycountry19802010millions.csv")
population = population.withColumnRenamed("_c0", "country")
ms = StructType([
    StructField("Year", IntegerType(), True),
    StructField("Country", StringType(), True),
    StructField("Drop", DoubleType(), True)])
D = spark.createDataFrame([],ms)

for year in range (1980, 2010):
    # Find PerDrop and sort by PerDrop in descending
    A = population.withColumn("PerDrop", (col(str(year)) - col(str(year+1)))*100/col(str(year))).sort(desc("PerDrop"))
    B = A.select(first("country").alias("Country"), (-1*first("PerDrop")).alias("Drop"))
    C = B.withColumn("Year", lit(year+1))
    D = D.union(C.select("Year", "Country","Drop"))
D.toPandas()


,Year,Country,Drop
0,1981,Afghanistan,-9.106331
1,1982,Afghanistan,-8.017227
2,1983,Antigua and Barbuda,-3.514189
3,1984,Antigua and Barbuda,-1.752514
4,1985,Cook Islands,-1.409245
5,1986,Netherlands Antilles,-24.587817
6,1987,Saint Helena,-21.299639
7,1988,Mozambique,-2.883632
8,1989,Somalia,-2.196497
9,1990,Liberia,-12.816300


In [3]:
D.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Drop: double (nullable = true)

